```
#!/bin/bash
DOMAIN=${1:-voronenko.au.auth0.com}
KONG_API=http://localhost:8001

PUBLIC_KEY=$(curl -sS https://${DOMAIN}/pem | openssl x509 -pubkey -noout)
[[ -z $PUBLIC_KEY ]] && echo "ERROR: could not retrieve public key of $PUBLIC_KEY" && exit 1

# add user
curl -sS -i -X POST $KONG_API/consumers --data username=$DOMAIN

# add public key to consumer
curl -i -X POST $KONG_API/consumers/$DOMAIN/jwt \
    -F "algorithm=RS256" \
    -F "rsa_public_key=${PUBLIC_KEY}" \
    -F "key=https://$DOMAIN/" 

# add an sample API
curl -i -X POST $KONG_API/apis \
    --data "name=headers" \
    --data "hostnames=httpbin.org" \
    --data "upstream_url=http://httpbin.org/anything" 

# add the JWT plugin on the Sample API
curl -i -X POST $KONG_API/apis/headers/plugins \
    --data name=jwt
```